# validating model performance

In [8]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification

In [9]:
test_model = BertForSequenceClassification.from_pretrained('./fine_tuned_bert_sentiment')
test_tokenizer = BertTokenizer.from_pretrained('./fine_tuned_bert_sentiment')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Function to predict sentiment
def predict_sentiment(text):
    inputs = test_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    # Move input tensors to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = test_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return "Positive" if probabilities[0][1] > probabilities[0][0] else "Negative"

In [5]:
# Check if CUDA is available and move model to GPU if possible
import textwrap

test_model = test_model.to(device)


print("\nNow, let's try with your own input!")
user_review = input("Enter a restaurant review: ")
user_sentiment = predict_sentiment(user_review)

prettified_review = textwrap.fill(user_review, width=80)
# print(f'\nYour review: {user_review}')
print(f"\nYour review:\n{'='*80}\n{prettified_review}\n{'='*80}")
print(f"\nPredicted sentiment for your review: {user_sentiment}")


Now, let's try with your own input!

Your review:
If you love dining in near darkness and squinting at a menu, The Gourmet Shack
is for you! The mismatched furniture adds that "we tried" vibe, and the service?
Practically invisible—until they swoop in like ninjas with your food.  Speaking
of the food, each dish looks like modern art—tiny portions that leave you
wondering if you've actually eaten anything. But hey, it's surprisingly
delicious. And the dessert? Perfectly sized for ants.  So, if you're into paying
for confusion and unexpectedly great flavors, this is the place for you!

Predicted sentiment for your review: Positive


c:\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


# Metrics validation